# answer question originated from unifying process

In [2]:
import pandas as pd
import numpy as np
import geopandas
import matplotlib.pyplot as plt
from pandas.io.parsers import read_csv
from gis_functions import *



# check original PE
## compare original data with half-way

In [ ]:
# why has karn so much Pe in plus?
import functions
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA Kärnten NEU.xlsx'

data, trial = functions.reader(d, 'Data K', header=15, test=False)
data['EW 60']=data['EW 60'].replace('?', 0)
data['EW 60']=data['EW 60'].astype(str)
data.loc[data['EW 60'].str.contains('Bew'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('Bescheid'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('m³'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('Liter'), 'EW 60']='0'
data.loc[data['EW 60'].str.contains('nan'), 'EW 60']='0'

#perfect match. carintia had from beginning so many PEs
data['EW 60'].astype(float).sum()

In [ ]:
#check original PE for LowerA
d = r'C:/Users/fabrizio/Documents/R_data/PROJECTS/MASTER/DATA/KKA NÖ NEU DE final.xlsx'
data, trial = functions.reader(d, 'Data NÖ (Technol)', header=5, test=False)

#again, perfect match
data['EW60_BEWILLIGT'].astype(float).sum()

# check the unmerged 
## why some rows have not been merged?

In [ ]:
# check the not merged of KArn
data=pd.read_excel('final/not_merged_karn.xlsx')

In [ ]:
#zoom on merge key
data.loc[:,['KG', 'KG_NR']]
#one KG , and the rest is 0 that used to be NAs
#OK, differenced between half-way and final are explained

In [ ]:
#all 0 except one
data=pd.read_excel('final/not_merged_sbg.xlsx')
data.loc[:,['KG', 'KG_NR']]


# unify OEBO

In [ ]:
oebo=geopandas.read_file('final/oebo.geojson')


In [ ]:
oebo['no_nitri']=oebo.freq-oebo.NITRIFIZIERUNG

# what the fuck happened with mean_age steiermark?

In [ ]:
steyr=pd.read_excel('half-way/steyr.xlsx')

In [ ]:
steyr.sort_values('year', ascending=True)## 2077
#steyr[steyr.sort_values('year', ascending=True).year>0]## 


In [ ]:
data=geopandas.read_file('final/steyr.geojson')


In [ ]:
data[data.sort_values('mean_year', ascending=True).mean_year>0]## 


# alle the 0s disturb the mean value!! where do 0 come from ? NAs

# inserting new function : sumPE_nonitri

In [10]:
data=geopandas.read_file('final/steyr.geojson')



In [11]:
data.columns

Index(['id', '%before_reg', '%no_nitri', 'BKZ', 'BL', 'BL_KZ', 'FA', 'FA_NR',
       'FL', 'GB', 'GB_KZ', 'GKZ', 'KG', 'KG_NR', 'MERIDIAN', 'PB', 'PG', 'ST',
       'ST_KZ', 'VA', 'VA_NR', 'before_reg', 'freq', 'mean_year', 'no_nitri',
       'sum_PE', 'geometry'],
      dtype='object')

In [12]:
data

,id,%before_reg,%no_nitri,BKZ,BL,BL_KZ,FA,FA_NR,FL,GB,...,ST,ST_KZ,VA,VA_NR,before_reg,freq,mean_year,no_nitri,sum_PE,geometry
0,0,0.000000,0.000000,614,Steiermark,6,Judenburg Liezen,71,8871328,Murau,...,Österreich,1,Judenburg,65,0,1,2008,0,6,"POLYGON ((14.14081 47.17795, 14.14088 47.17862..."
1,1,0.000000,0.000000,617,Steiermark,6,Oststeiermark,67,24470041,Weiz,...,Österreich,1,Weiz,68,0,16,2007,0,192,"POLYGON ((15.58239 47.41321, 15.58229 47.41347..."
2,2,0.000000,0.000000,610,Steiermark,6,Deutschlandsberg Leibnitz Voitsberg,72,4854117,Leibnitz,...,Österreich,1,Leibnitz,66,0,1,2002,0,12,"POLYGON ((15.55413 46.86375, 15.55396 46.86429..."
3,3,0.000000,0.000000,622,Steiermark,6,Oststeiermark,67,4860531,Fürstenfeld,...,Österreich,1,Weiz,68,0,1,2009,0,10,"POLYGON ((15.80463 47.46098, 15.80484 47.46097..."
4,4,100.000000,100.000000,622,Steiermark,6,Oststeiermark,67,5245035,Fürstenfeld,...,Österreich,1,Weiz,68,3,3,1962,3,50,"POLYGON ((15.90559 47.21368, 15.90552 47.21365..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1236,1236,100.000000,100.000000,620,Steiermark,6,Judenburg Liezen,71,4523059,Judenburg,...,Österreich,1,Judenburg,65,1,1,1963,1,5,"POLYGON ((14.80682 47.20990, 14.80683 47.20994..."
1237,1237,27.272727,27.272727,616,Steiermark,6,Deutschlandsberg Leibnitz Voitsberg,72,6362950,Voitsberg,...,Österreich,1,Graz,63,3,11,1999,3,71,"POLYGON ((15.09584 47.02277, 15.09587 47.02293..."
1238,1238,20.000000,20.000000,623,Steiermark,6,Oststeiermark,67,6743014,Feldbach,...,Österreich,1,Feldbach,62,1,5,1998,1,60,"POLYGON ((15.92159 46.99260, 15.92159 46.99262..."
1239,1239,100.000000,100.000000,601,Steiermark,6,Graz-Stadt,68,946822,Graz-Ost,...,Österreich,1,Graz,63,1,1,1965,1,4,"POLYGON ((15.45578 47.02151, 15.45565 47.02174..."


In [9]:

data[['%PE_no_nitri','PE_nonitri', 'freq','no_nitri', 'sum_PE']].sort_values('%PE_no_nitri',ascending=False)

,%PE_no_nitri,PE_nonitri,freq,no_nitri,sum_PE
44,40,21,43,40,152
886,40,27,44,39,342
142,34,8,45,32,150
95,34,10,46,32,201
162,33,9,41,28,145
...,...,...,...,...,...
563,0,0,8,0,63
565,0,0,56,0,549
568,0,0,5,0,28
571,0,0,1,0,8
